In [ ]:
# pip install accelerate

In [ ]:
# switch to gpu if possible
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

device

In [ ]:
# Get the current memory usage and the maximum memory usage in bytes
current_memory_allocated = torch.cuda.memory_allocated(device)
max_memory_allocated = torch.cuda.max_memory_allocated(device)

# Convert bytes to a more readable format, like MB
current_memory_mb = current_memory_allocated / (1024**2)
max_memory_mb = max_memory_allocated / (1024**2)

print(f"Current Memory Allocated: {current_memory_mb} MB")
print(f"Max Memory Allocated: {max_memory_mb} MB")

# Get overall memory usage statistics
total_memory = torch.cuda.get_device_properties(device).total_memory
free_memory = torch.cuda.memory_reserved(device) - torch.cuda.memory_allocated(device)


print(f"Total Memory: {total_memory / (1024**2)} MB")

## Test mistral 7B

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("rishiraj/CatPPT")
tokenizer = AutoTokenizer.from_pretrained("rishiraj/CatPPT")

In [ ]:
model.to(device)

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model="rishiraj/CatPPT",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
prompt = pipe.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
outputs = pipe(
    prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95
)
print(outputs[0]["generated_text"])

In [ ]:
# def call_mistral(text):
messages = [{"role": "user", "content": "hi there"}]
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

decoded = tokenizer.batch_decode(generated_ids)

# Split the decoded text on the [/INST] token and return the part after it
response = decoded[0].split("[/INST]")[-1].strip()

# Further trim the "</s>" token from the end of the response if it exists
if response.endswith("</s>"):
    response = response[:-4].strip()

In [ ]:
response = call_mistral("hi, are you a consious entity?")

In [ ]:
print(response)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "can you write me python code for a simple game such as black jack, like with classes, etc. so i can simulate the game and estimate the odds to win given a certain hand. ",
    }
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

In [ ]:
model_inputs = encodeds.to(device)
model.to(device)

In [ ]:
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

In [ ]:
generated_ids

In [ ]:
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

In [ ]:
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

In [ ]:
df_cc.iloc[12]["extr_text"]  # asml as test